# Build an Image Classification Model

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore",category=UserWarning)

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Dense,Flatten,Conv2D, MaxPooling2D
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical

# Load Data

In [42]:
df1 = pd.read_csv("train_data.csv")

In [43]:
df1

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_3063,pixel_3064,pixel_3065,pixel_3066,pixel_3067,pixel_3068,pixel_3069,pixel_3070,pixel_3071,label
0,59,62,63,43,46,45,50,48,43,68,...,216,184,140,151,118,84,123,92,72,6
1,154,177,187,126,137,136,105,104,95,102,...,143,133,139,143,134,142,143,133,144,9
2,255,255,255,253,253,253,253,253,253,253,...,78,85,83,79,85,83,80,86,84,9
3,28,25,10,37,34,19,38,35,20,42,...,54,47,28,63,56,37,72,65,46,4
4,170,180,198,168,178,196,177,185,203,183,...,75,79,82,71,75,78,73,77,80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,145,161,194,151,166,199,156,171,205,155,...,39,41,56,36,38,53,37,39,54,2
49996,255,245,132,254,247,134,255,249,137,254,...,249,174,79,252,181,81,253,181,76,6
49997,35,178,235,40,176,239,42,176,241,39,...,27,52,77,21,43,66,12,31,50,9
49998,189,211,240,186,208,236,185,207,235,184,...,195,184,169,196,189,171,195,190,171,1


In [44]:
X_train = df1.drop("label", axis=1)
Y_train = df1["label"]

In [45]:
df2 = pd.read_csv("test_data.csv")

In [46]:
df2

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_3063,pixel_3064,pixel_3065,pixel_3066,pixel_3067,pixel_3068,pixel_3069,pixel_3070,pixel_3071,label
0,158,112,49,159,111,47,165,116,51,166,...,24,77,124,34,84,129,21,67,110,3
1,235,235,235,231,231,231,232,232,232,232,...,168,183,178,180,195,191,186,200,199,8
2,158,190,222,158,187,218,139,166,194,132,...,5,6,8,4,5,3,7,8,7,8
3,155,156,149,167,176,187,176,179,193,190,...,60,63,50,64,65,52,73,68,50,0
4,65,68,50,70,81,64,48,64,46,30,...,143,179,136,154,185,146,128,156,117,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,76,78,133,76,78,134,77,80,132,76,...,204,219,236,200,215,234,199,216,236,8
9996,81,57,43,91,69,53,98,75,63,106,...,155,139,129,171,167,147,167,164,160,3
9997,20,15,12,19,14,11,15,14,11,15,...,24,23,50,26,23,53,25,20,47,5
9998,25,40,12,15,36,3,23,41,18,17,...,93,124,80,95,123,81,92,120,80,1


In [47]:
X_test = df2.drop('label', axis=1)

In [48]:
X_train.shape

(50000, 3072)

In [49]:
X_test.shape

(10000, 3072)

In [50]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [51]:
Y_train = np.array(Y_train)

In [52]:
X_train = X_train.reshape(X_train.shape[0],32,32,3)
X_test = X_test.reshape(X_test.shape[0],32,32,3)

In [53]:
X_train.shape

(50000, 32, 32, 3)

In [54]:
X_train

array([[[[ 59,  62,  63],
         [ 43,  46,  45],
         [ 50,  48,  43],
         ...,
         [158, 132, 108],
         [152, 125, 102],
         [148, 124, 103]],

        [[ 16,  20,  20],
         [  0,   0,   0],
         [ 18,   8,   0],
         ...,
         [123,  88,  55],
         [119,  83,  50],
         [122,  87,  57]],

        [[ 25,  24,  21],
         [ 16,   7,   0],
         [ 49,  27,   8],
         ...,
         [118,  84,  50],
         [120,  84,  50],
         [109,  73,  42]],

        ...,

        [[208, 170,  96],
         [201, 153,  34],
         [198, 161,  26],
         ...,
         [160, 133,  70],
         [ 56,  31,   7],
         [ 53,  34,  20]],

        [[180, 139,  96],
         [173, 123,  42],
         [186, 144,  30],
         ...,
         [184, 148,  94],
         [ 97,  62,  34],
         [ 83,  53,  34]],

        [[177, 144, 116],
         [168, 129,  94],
         [179, 142,  87],
         ...,
         [216, 184, 140],
        

In [55]:
X_test.shape

(10000, 32, 32, 3)

In [56]:
lb=LabelBinarizer()
Y_train = lb.fit_transform(Y_train)

# Build and Train Model

In [57]:
model = Sequential()

In [58]:
model.add(Conv2D(64, (3,3), input_shape=(32,32,3), activation="relu"))
model.add(MaxPooling2D((3,3)))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(16, (2,2), activation="relu"))
model.add(Flatten())
model.add(Dense(64, activation="sigmoid"))
model.add(Dense(10, activation="softmax"))

In [59]:
model.compile(loss= "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [60]:
e = 5
H = model.fit(X_train, Y_train, validation_data=(X_train,Y_train), epochs = e, batch_size=32)

Epoch 1/5
1560/1563 [============================>.] - ETA: 0s - loss: 2.0191 - accuracy: 0.2321

MemoryError: Unable to allocate 1.14 GiB for an array with shape (50000, 32, 32, 3) and data type int64

# Plot Loss and Accuracy

In [ ]:
plt.style.use("ggplot")
plt.plot(np.arange(0,e), H.history["loss"], label="loss")
plt.plot(np.arange(0,e), H.history["accuracy"],label="accuracy")
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.title("Training Loss and Accuracy")
plt.legend()

In [ ]:
Y_pred = model.predict(X_test, batch_size=32)

In [ ]:
Y_pred.shape

In [ ]:
# Y_pred = Y_pred.argmax(axis=1)

In [ ]:
# Y_pred.shape

In [ ]:
Y_pred

In [ ]:
Y_train
Y_test = df1["label"]
Y_test = np.array(Y_test)
Y_test

In [ ]:
accuracy_score(Y_pred.argmax(axis=1), Y_test.argmax(axis=1))